<a href="https://colab.research.google.com/github/tsubauaaa/Tensorflow2.0-tutorials/blob/master/Tensorflow2_model_save_and_restore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## モデルの保存と復元

モデルは訓練中にも、訓練が終わったあとも保存できる。このことは、長い訓練時間を掛けなくても、やめたところから再開できるということを意味する。モデルが保存可能であることは、あなたが作ったモデルを他の人と共有できるということでもある。研究結果であるモデルや手法を公開する際、機械学習の実践者はほとんど次のものを共有する。

* モデルを構築するプログラム
* 学習済みモデルの重みあるいはパラメータ

このデータを共有することで、他の人がモデルをどの様に動作するかを理解したり、新しいデータに試してみたりすることが容易になる。

### オプション
TensorFlowのモデルを保存する方法は、使っているAPIによって異なる。このガイドはTensorFlowのモデルを構築し訓練するためのハイレベルなAPIであるtf.kerasを使っている。

In [1]:
!pip install -q h5py pyyaml
!pip install tensorflow-gpu==2.0.0

### サンプルデータセットの取得
ここでは、モデルを訓練し重みの保存をデモするために、 MNIST dataset を使う。デモの実行を速くするため、最初の1,000件のサンプルだけを使う。

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### モデルの定義
重みの保存と読み込みのデモを行うための簡単なモデルを定義する。

In [4]:
# 短いシーケンシャルモデルを返す関数
def create_model():
  model = tf.keras.models.Sequential([
      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model


# 基本的なモデルのインスタンスを作る
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 訓練中にチェックポイントを保存する
主な用途は訓練の途中あるいは終了後にチェックポイントを自動的に保存すること。こうすることにより、再び訓練を行うことなくモデルを使用することができ、また、訓練が中断された場合に、中止したところから再開できる。

tf.keras.callbacks.ModelCheckpointがこれを行うためのコールバック。このコールバックにはチェックポイントを構成するためのいくつかの引数がある。

#### チェックポイントコールバックの使い方
モデルの訓練時に、ModelCheckpointを渡す。

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                                                       save_weights_only=True,
                                                                                       verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback]) # 訓練にコールバックを渡す

# オプティマイザの状態保存についての警告が表示されるかもしれない。
# これらの警告は（このノートブックで発生する同様な警告を含めて）
# 古い用法を非推奨にするためのもので、無視してOK。

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.3045 - accuracy: 0.5950
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 996us/sample - loss: 1.1779 - accuracy: 0.6410 - val_loss: 0.7237 - val_accuracy: 0.7830
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4279 - accuracy: 0.8862
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 306us/sample - loss: 0.4259 - accuracy: 0.8820 - val_loss: 0.5369 - val_accuracy: 0.8350
Epoch 3/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2668 - accuracy: 0.9330
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 303us/sample - loss: 0.2715 - accuracy: 0.9310 - val_loss: 0.4727 - val_accuracy: 0.8570
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2010 - accuracy: 0.9483
Epoch

この結果、エポックごとに更新される一連のTensorFlowチェックポイントファイルが作成される。

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


訓練していない新しいモデルを作る。重みだけからモデルを復元する場合には、元のモデルと同じアーキテクチャのモデルが必要。モデルのアーキテクチャが同じであるため、モデルの異なるインスタンスであっても重みを共有することができる。

訓練していない全く新しいモデルを作り、テストデータセットで評価する。訓練をしていないモデルは偶然のレベル（正解率10%以下）の性能しか無いはず。

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4882 - accuracy: 0.0820
Untrained model, accuracy:  8.20%


次に、チェックポイントから重みをロードし、再び評価する。

In [8]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4213 - accuracy: 0.8640
Untrained model, accuracy: 86.40%


### チェックポイントコールバックのオプション
このコールバックには、チェックポイントに一意な名前をつけたり、チェックポイントの頻度を調整するためのオプションがある。

新しいモデルを訓練し、5エポックごとに一意な名前のチェックポイントを保存する。

In [9]:
# ファイル名に(`str.format`を使って)エポック数を埋め込む
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 重みを5エポックごとに保存する
    period=5)

model = create_model()
model.fit(train_images, train_labels,
                 epochs = 50, callbacks = [cp_callback],
                 validation_data = (test_images, test_labels),
                 verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


次に、出来上がったチェックポイントを確認し、最後のものを選択する。

In [10]:
!ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt
cp-0005.ckpt			  cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt
cp-0010.ckpt			  cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt
cp-0015.ckpt			  cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt
cp-0020.ckpt			  cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt
cp-0025.ckpt			  cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001  cp-0050.ckpt.index
cp-0025.ckpt.index


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

注意：デフォルトのtensorflowフォーマットは、直近の5つのチェックポイントのみを保存する。

テストのため、モデルをリセットし最後のチェックポイントをロードする。

In [14]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5353 - accuracy: 0.8820
Restored model, accuracy: 88.20%


### これらのファイルは何？
上記のコードでは、重みだけをバイナリでcheckpoint形式の一連のファイルに保存する。チェックポイントには、次のものが含まれる。

* １つ以上のモデルの重みの断片
* どの重みがどの断片に保存されているかを示すインデックスファイル

1台のマシンだけでモデルの訓練を行っている場合には.data-00000-of-00001のようなサフィックスのついたファイルが1つだけ作成される。

### 手動で重みを保存する
上記では重みをモデルにロードする方法を見た。

手動で重みを保存するのも同じ様に簡単。model.save_weights メソッドを使う。

In [15]:
# 重みの保存
model.save_weights('./checkpoints/my_checkpoint')

# 重みの復元
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5353 - accuracy: 0.8820
Restored model, accuracy: 88.20%


### モデル全体の保存
モデルとオプティマイザを、その状態（重みと変数）とモデルの設定の両方を含む1つのファイルに保存することができる。これにより、モデルをオリジナルのPythonコードにアクセスしなくとも使用できるようにエクスポートできる。オプティマイザの状態が復元されるので、中断したところから訓練を再開することも可能。

完全に機能するモデルを保存できるのは便利。保存したモデルをTensorFlow.js (HDF5, Saved Model)でロードし、ブラウザで訓練したり、実行したりすることができるほか、TensorFlow Lite (HDF5, Saved Model) を使ってモバイルデバイスで実行できるように変換することも可能。

HDF5ファイルとして
Kerasでは、HDF5 標準を使った基本的なファイルフォーマットが利用できる。ここでの利用目的では、保存されたモデルは単独のバイナリラージオブジェクト（blob）として扱うことができる。

In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# モデル全体を1つのHDF5ファイルに保存する。
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 638us/sample - loss: 1.1314 - accuracy: 0.6950
Epoch 2/5
1000/1000 [==============================] - 0s 159us/sample - loss: 0.4233 - accuracy: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 173us/sample - loss: 0.2733 - accuracy: 0.9330
Epoch 4/5
1000/1000 [==============================] - 0s 175us/sample - loss: 0.2134 - accuracy: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 160us/sample - loss: 0.1622 - accuracy: 0.9630


保存したファイルを使ってモデルを再作成する。

In [17]:
# 重みとオプティマイザを含む全く同じモデルを再作成
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


正解率を検査する。

In [18]:
loss ,acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4646 - accuracy: 0.8560
Restored model, accuracy: 85.60%


この方法では、次のすべてが保存される。

* 重みの値
* モデルの設定（アーキテクチャ）
* オプティマイザの設定

Kerasは保存する際にアーキテクチャを調べる。いまのところ、TensorFlowのオプティマイザ（tf.trainに含まれるもの）を保存することはできない。TensorFlowのオプティマイザを使用している場合には、モデルをロードしたあと再コンパイルする必要があり、オプティマイザの状態は失われる。

saved_modelとして
注意：この手法によるtf.kerasモデルの保存は実験的なもので、将来のバージョンで変更される可能性がある。

新しいモデルを作る。

In [19]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 495us/sample - loss: 1.2101 - accuracy: 0.6600
Epoch 2/5
1000/1000 [==============================] - 0s 172us/sample - loss: 0.4319 - accuracy: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 175us/sample - loss: 0.2878 - accuracy: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 174us/sample - loss: 0.2122 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 180us/sample - loss: 0.1615 - accuracy: 0.9680


saved_modelを作成し、タイムスタンプ付きのディレクトリに保存する。

In [20]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

'./saved_models/1570608334'

作成したsaved_modelsを一覧表示する。

In [21]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


復元されたモデルを実行する。

In [23]:
model.predict(test_images).shape

(1000, 10)

In [24]:
# モデルを評価する前にコンパイルする必要がある。
# モデルをデプロイするだけであればこのステップは不要。

new_model.compile(optimizer=model.optimizer, # ロードしてあったオプティマイザを保持
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# モデルを評価する。
loss , acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))



1000/1 - 0s - loss: 0.5804 - accuracy: 0.8630
Restored model, accuracy: 86.30%
